# Clustering

Clustering is an example of unsupervised learning in which we work with completely unlabeled data (or in which our data has labels but we ignore them)

- The clusters won't label themselves. You'll have to do that by looking at the data underlying each one.

One of the simplest clustering methods is `k-means`

In which the number of clusters **k** is choosen in advance

`k = number of clusters we want`

Then the goal is:
- Divide the data into k groups so that points in each group are close to that group’s average (called the mean).

We try to minimize:

- The total squared distance between each point and the mean of its cluster


There are a lot of ways to assign **n** points to **k** clusters, which means that finding an optimal clustering is a very hard problem.
We will settle for an iterative algorithm that usually finds a good clustering:

- 1. Start with a set of **k-means**, which are points in d-dimensional space
- 2. Assign each point to the mean to which it is closed
- 3. If no point's assignment has changed, stop and keep the clusters
- 4. If some point's assignment has changed, recompute the means and returns to step 2


In [ ]:
# Measures how many coordinates two vectors differ in.

from typing import List

Vector = List[float]

def num_differences(v1:Vector, v2:Vector) -> int:
    assert len(v1) == len(v2)
    return len([x1 for x1,x2 in zip(v1,v2) if x1 != x2])

assert num_differences([1,2,3],[2,1,3])
assert num_differences([1,2],[1,2]) == 0

We also need a function that, given some vectors and their assignment to clusters, computes the means of the clusters. It may be the case that some cluster has no points to assigned to it. We can't take the mean of an empty collection, so in this case we'll just randomly pick one of the points to serve as the "mean" of that cluster

In [3]:
from typing import List
import random

def vector_sum(vectors: List[Vector]) -> Vector:
    # check that vectors is not empty
    assert vectors, "no vectors provided!"

    # check the vectors are all the same size
    num_elements = len(vectors[0])
    assert all(len(v) == num_elements for v in vectors), "different sizes"
    return [sum(vector[i] for vector in vectors) for i in range(num_elements)]

def vector_mean(vectors: List[Vector])-> Vector:
    num_elements = len(vectors)
    return [(1/num_elements)*i for i in vector_sum(vectors)]

def cluster_means(k: int, inputs: List[Vector], assignments: List[int]) -> List[Vector]:
    clusters = [[] for i in range(k)]

    for input, assignment in zip(inputs, assignments):
        clusters[assignment].append(input)

    # If a cluster is empty, we just use a random point
    return [vector_mean(cluster) if cluster else random.choice(inputs) for cluster in clusters]  

We will use **tqdm** to track our progress but here we don't know how many iterations it will take. So we then use **itertools.count**, which creates an infinite iterable adn we will return out it when we are done.

In [4]:
import itertools
import random
import tqdm

def dot(v:Vector,w:Vector)->Vector:
    assert len(v) == len(w), "vectors must be of same length"
    return sum(v_i*w_i for v_i,w_i in zip(v,w))

def sum_of_squares(v:Vector)->float:
    return dot(v,v)

def subtract(v:Vector,w:Vector):
    assert len(v) == len(w)
    return[v_i-w_i for v_i, w_i in zip(v,w)]

def squared_distance(v:Vector, w:Vector) -> float:
    return sum_of_squares(subtract(v,w))

class KMeans:
    def __init__(self, k:int) -> None:
        self.k = k             # number of clusters
        self.means = None

    def classify(self, input: Vector) -> int:
        """ Return the index of the cluster closet to the input """
        return min(range(self.k), key=lambda i: squared_distance(input, self.means[i]))
    
    def train(self, inputs: List[Vector]) -> None:
        # Start with random assignments 
        assignments  = [random.randrange(self.k) for _ in inputs]

        with tqdm.tqdm(itertools.count()) as t:
            for _ in t:
                # Compute means and find new assignments
                self.means = cluster_means(self.k, inputs, assignments)
                new_assignments = [self.classify(input) for input in inputs]

                # check how many assignments changed and if we are done
                num_changed = num_differences(assignments, new_assignments)
                if num_changed == 0:
                 return
                
                # Otherwise keep the new assignments and compute the new means
                assignments = new_assignments
                self.means = cluster_means(self.k, inputs, assignments)
                t.set_description(f"changed: {num_changed} / {len(inputs)}")
        

## Choosing (k)

There are various ways to choose **k**. One that is resonably easy to understand involves plotting the **sum of squared errors**(between each point and the mean of its cluster) as function of **k** and looking at where the graph "bends". Also known as `Elbow Method`

In [ ]:
from matplotlib import pyplot as plt

def squared_clustering_errors(inputs: List[Vector], k:int) -> float:
    """ Finds the total squared error from k-means clustering the inputs """
    clusterer = KMeans(k)
    clusterer.train(inputs)
    means = clusterer.means
    assignments = [clusterer.classify(input) for input in inputs]

    return sum(squared_distance(input,means[cluster]) for input, cluster in zip(inputs, assignments))

# now plot from 1 up to len(inputs) clusters

ks = range(1,len(inputs)+1)
errors = [squared_clustering_errors(inputs,k) for k in ks]

plt.plot(ks,errors)
plt.xticks(ks)
plt.xlabel("k")
plt.ylabel("total squared error")
plt.title("Total Error vs. # of Clusters")
plt.show()

## Example: Clustering Colors

Computer images can be represented as two-dimensional arrays of pixels, where each pixel is itself a three-dimensional vector(red, green, blue) indicating its color

Image as a 2D Array

If an image is:

- Height = 3 pixels
- Width = 4 pixels

It looke like:

[
  [[R,G,B], [R,G,B], [R,G,B], [R,G,B]],

  [[R,G,B], [R,G,B], [R,G,B], [R,G,B]],

  [[R,G,B], [R,G,B], [R,G,B], [R,G,B]]
]


Creating a five-color version of the image , then entails:

 1. Choosing five colors.
 2. Assigning one of those colors to each pixel

To start with, we need a way to load image into Python. We can do this  with matplotlib, if  we first install the pillow library:

`pip install pillow`

In [6]:
! pip install pillow


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
image_path = r"girl_with_book.jpg"
import matplotlib.image as mpimg
img = mpimg.imread(image_path)/256                  # Rescale to between 0 and 1 

Behind the scenes **img** is a **NumPy** array, but for our purposes, we can treat it as a list of lists

`img[i][j]` is the pixel in the ith rowa nd jth column, each pixel is the list of `[red, green, blue]` of numbers between 0 and 1 indicating the color of that pixel

In [ ]:
top_row = img[0]
top_left_pixel = top_row[0]
red, green, blue = top_left_pixel

# We can get flattened list of all the pixels
# .tolist() converts a Numpy array to a python list

pixels = [pixel.tolist() for row in img for pixel in row]

# Now feed them to our clusters

clusterer  = KMeans(5)
clusterer.train(pixels)   # this might take a while

# Once it finishes, We just construct a new image with the same format
def recolor(pixel: Vector) -> Vector:
    cluster = clusterer.classify(pixel)         # index of the closest cluster
    return clusterer.means[cluster]             # mean of the closes cluster

new_img = [[recolor(pixel) for pixel in row] for row in img]

# display it, using plt.imshow
plt.imshow(new_img)
plt.axis('off')
plt.show()

## Bottom-Up Hierarchical Clustering

An alternative approach to clustering is to "grow" clusters from the bottom up.

We can do this in the following way:

 1. Make each input its own cluster of one.
 2. As long as there are multiple clusters remaining, find the two closest clusters and merge them.

In [4]:
# Our values will live in leaf clusters

from typing import NamedTuple, Union,List
Vector = List[float]

class Leaf(NamedTuple):
    value: Vector

leaf1 = Leaf([10, 20])
leaf2 = Leaf([30, -15])


class Merged(NamedTuple):
    children: tuple
    order: int

merged = Merged((leaf1,leaf2), order=1)

Cluster = Union[Leaf, Merged]

#A cluster can be either:
#A Leaf (single point)
# OR a Merged cluster (group of clusters)

In [5]:
# Helper function that recursively return all the values contained in a (possible merged) cluster

def get_values(cluster: Cluster) -> List[Vector]:
    if isinstance(cluster, Leaf):
        return [cluster.value]
    
    else:
        return [value 
                for child in cluster.children
                for value in get_values(child)]
    
assert get_values(merged) == [[10, 20], [30, -15]]

In order to merge the closest clusters, we need some notion of the distance between clusters.

1. Minimum distance (Single Linkage)
2. Maximun distance (Complete Linkage)
3. Average distance

In [6]:
from typing import Callable
import math


def distance(v:Vector, w:Vector) -> float:
    return math.sqrt(squared_distance(v,w)) 

def cluster_distance(cluster1: Cluster, 
                     cluster2: Cluster, 
                     distance_agg: Callable = min) -> float: 
    """
    compute all the pairwise distances between cluster1 and cluster2
    and apply the aggregation function _distance_agg_ to the resulting list
    """ 
    return distance_agg([distance(v1, v2) 
                         for v1 in get_values(cluster1) 
                         for v2 in get_values(cluster2)])

We will use merge order slot to track the order.

Since **Leaf** clusters were never merged, we will assign them infinity the highest possible value.

In [7]:
def get_merge_order(cluster: Cluster) -> float:
    if isinstance(cluster, Leaf):
        return float('inf')      # was never merged
    else:
        return cluster.order

Since **Leaf** clusters don't have children, we will create and add a helper function

In [9]:
from typing import Tuple

def get_children(cluster: Cluster):
    if isinstance(cluster,Leaf):
        raise TypeError("Leaf has no children")
    else:
        return cluster.children

Creating the clustering algorithm

In [ ]:
def bottom_up_cluster(inputs: List[Vector], distance_agg: Callable = min) -> Cluster:
    
    # Start with all leaves
    clusters: List[Cluster] = [Leaf(input) for input in inputs]

    def pair_distance(pair: Tuple[Cluster, Cluster]) -> float:
        return cluster_distance(pair[0],pair[1],distance_agg)
    
    # as long as we have more than one cluster left...
    while len(clusters) > 1:
        # find two closest clusters
        c1,c2 = min(((cluster1,cluster2) for i,cluster1 in enumerate(clusters)
                    for cluster2 in clusters[:i]), key=pair_distance)
        
        # remove them from the list of clusters
        clusters = [c for c in clusters if c!= c1 and c != c2]

        # merge them using merge_order = # of clusters left
        merged_cluster = Merged((c1,c2), order = len(clusters))

        # and add their merge
        clusters.append(merged_cluster)
        
    # when there's only one cluster left return it
    return clusters[0]

Function that generates any numebr fo clusters by performing the appropriate number of unmerges

In [ ]:
def generate_clusters(base_cluster: Cluster, 
                      num_clusters: int) -> List[Cluster]: 
    # start with a list with just the base cluster 
    clusters = [base_cluster] 
 
    # as long as we don't have enough clusters yet... 
    while len(clusters) < num_clusters: 
        # choose the last-merged of our clusters 
        next_cluster = min(clusters, key=get_merge_order) 
        # remove it from the list 
        clusters = [c for c in clusters if c != next_cluster] 
 
        # and add its children to the list (i.e., unmerge it) 
        clusters.extend(get_children(next_cluster)) 
 
    # once we have enough clusters... 
    return clusters

For example, if we want to generate three clusters we can just do 

In [ ]:
base_cluster = bottom_up_cluster(inputs)

three_clusters = [get_values(cluster) for cluster in generate_clusters(base_cluster,3 )]